In [103]:
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
import time
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import re
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [104]:
chromedriver = '/Users/dc/practice/recommendation/recomm_drf/cafe/chromedriver'
#chromedriver = '/Users/dc/study_dir/recommendation/cafe/chromedriver'
driver = webdriver.Chrome(chromedriver) 

/var/folders/yv/383sshfd5vnd2ylx0jvq9y3c0000gn/T/ipykernel_90941/4251084422.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [105]:
df = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe', '성동구_카페 정보.csv'), sep=',')

In [93]:
#df = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe', '종로구_카페 정보.csv'), sep=',')

In [94]:
# df['dong'] = df.iloc[:, 3].str.split(' ').str[1]
# print(df['dong'])

In [95]:
# df.head(10)

In [106]:
df['place_name']

0          블루보틀 성수 카페
1              할아버지공장
2                  쎈느
3              어니언컴퍼니
4                대림창고
            ...      
670              브라운팟
671            플디 성수점
672         카페노티드 피치스
673              천상가옥
674    비엔나커피하우스 서울숲역점
Name: place_name, Length: 675, dtype: object

In [107]:
df['naver_keyword'] = df['place_name']
df['naver_map_url'] = ''
df.head(10)

,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,1492599844,블루보틀 성수 카페,서울 성동구 성수동1가 656-302,서울 성동구 아차산로 7,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/1492599844,블루보틀 성수 카페,
1,1,1301973155,할아버지공장,서울 성동구 성수동2가 309-133,서울 성동구 성수이로7가길 9,010-3934-2301,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1301973155,할아버지공장,
2,2,1783691580,쎈느,서울 성동구 성수동2가 314-2,서울 성동구 연무장5길 20,02-469-2122,카페,음식점 > 카페,http://place.map.kakao.com/1783691580,쎈느,
3,3,145791269,어니언컴퍼니,서울 성동구 성수동2가 277-135,서울 성동구 아차산로9길 8,02-1644-1941,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/145791269,어니언컴퍼니,
4,4,1128989913,대림창고,서울 성동구 성수동2가 322-32,서울 성동구 성수이로 78,02-499-9669,카페,음식점 > 카페 > 테마카페 > 갤러리카페,http://place.map.kakao.com/1128989913,대림창고,
5,5,286483883,센터커피 서울숲점,서울 성동구 성수동1가 685-478,서울 성동구 서울숲2길 28-11,070-8868-2008,카페,음식점 > 카페 > 커피전문점,http://place.map.kakao.com/286483883,센터커피 서울숲점,
6,6,1460620031,인더매스 마장,서울 성동구 마장동 776-1,서울 성동구 마장로 270,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1460620031,인더매스 마장,
7,7,547570065,옹근달,서울 성동구 성수동2가 333-52,서울 성동구 성수이로7길 41-1,02-499-5012,카페,음식점 > 카페,http://place.map.kakao.com/547570065,옹근달,
8,8,1842536906,멜로워 성수플래그쉽,서울 성동구 성수동2가 333-94,서울 성동구 성수이로7길 39,02-499-1112,카페,음식점 > 카페,http://place.map.kakao.com/1842536906,멜로워 성수플래그쉽,
9,9,1302549042,로우키,서울 성동구 성수동2가 302-16,서울 성동구 연무장3길 6,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1302549042,로우키,


In [108]:
for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print(f'이번에 찾을 키워드 : {i}행 / 전체 {df.shape[0] -1}행중 {keyword}')
    if ' ' in keyword:
        join_keyword = '%20'.join(keyword.split())
    else:
        join_keyword = keyword
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={join_keyword}&sm=hty&style=v5"
        
        print(f'네이버 지도 url 정보: {naver_map_search_url}')

        driver.get(naver_map_search_url)
        time.sleep(2.5)
        df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, '#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview').get_attribute('data-cid')
        
        time.sleep(1.0)
        
        #만약 검색 결과가 없다면?
    except Exception as e:
        print(e)
        pass
    
driver.quit()

이번에 찾을 키워드 : 0행 / 전체 674행중 블루보틀 성수 카페
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=블루보틀%20성수%20카페&sm=hty&style=v5
이번에 찾을 키워드 : 1행 / 전체 674행중 할아버지공장
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=할아버지공장&sm=hty&style=v5
이번에 찾을 키워드 : 2행 / 전체 674행중 쎈느
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=쎈느&sm=hty&style=v5
이번에 찾을 키워드 : 3행 / 전체 674행중 어니언컴퍼니
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=어니언컴퍼니&sm=hty&style=v5
이번에 찾을 키워드 : 4행 / 전체 674행중 대림창고
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=대림창고&sm=hty&style=v5
이번에 찾을 키워드 : 5행 / 전체 674행중 센터커피 서울숲점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=센터커피%20서울숲점&sm=hty&style=v5
이번에 찾을 키워드 : 6행 / 전체 674행중 인더매스 마장
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=인더매스%20마장&sm=hty&style=v5
이번에 찾을 키워드 : 7행 / 전체 674행중 옹근달
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=옹근달&sm=hty&style=v5
이번에 찾을 키워드

이번에 찾을 키워드 : 66행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 67행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&sm=hty&style=v5
이번에 찾을 키워드 : 68행 / 전체 674행중 맛차차
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=맛차차&sm=hty&style=v5
이번에 찾을 키워드 : 69행 / 전체 674행중 아이오쓰리 성수사거리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아이오쓰리%20성수사거리점&sm=hty&style=v5
이번에 찾을 키워드 : 70행 / 전체 674행중 브라운팟
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=브라운팟&sm=hty&style=v5
이번에 찾을 키워드 : 71행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 72행 / 전체 674행중 카페노티드 피치스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20피치스&sm=hty&style=v5
이번에 찾을 키워드 : 73행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style=v5
이번에 

이번에 찾을 키워드 : 131행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 132행 / 전체 674행중 카페노티드 피치스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20피치스&sm=hty&style=v5
이번에 찾을 키워드 : 133행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style=v5
이번에 찾을 키워드 : 134행 / 전체 674행중 비엔나커피하우스 서울숲역점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=비엔나커피하우스%20서울숲역점&sm=hty&style=v5
이번에 찾을 키워드 : 135행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 136행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 137행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 138행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/

이번에 찾을 키워드 : 195행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 196행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 197행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 198행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페온화%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 199행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 200행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20성수&sm=hty&style=v5
이번에 찾을 키워드 : 201행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 202행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&

이번에 찾을 키워드 : 259행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 260행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20성수&sm=hty&style=v5
이번에 찾을 키워드 : 261행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 262행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&sm=hty&style=v5
이번에 찾을 키워드 : 263행 / 전체 674행중 맛차차
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=맛차차&sm=hty&style=v5
이번에 찾을 키워드 : 264행 / 전체 674행중 아이오쓰리 성수사거리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아이오쓰리%20성수사거리점&sm=hty&style=v5
이번에 찾을 키워드 : 265행 / 전체 674행중 브라운팟
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=브라운팟&sm=hty&style=v5
이번에 찾을 키워드 : 266행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5


...

이번에 찾을 키워드 : 388행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style=v5
이번에 찾을 키워드 : 389행 / 전체 674행중 비엔나커피하우스 서울숲역점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=비엔나커피하우스%20서울숲역점&sm=hty&style=v5
이번에 찾을 키워드 : 390행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 391행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 392행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 393행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페온화%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 394행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 395행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.nav

이번에 찾을 키워드 : 452행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 453행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페온화%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 454행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 455행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20성수&sm=hty&style=v5
이번에 찾을 키워드 : 456행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 457행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&sm=hty&style=v5
이번에 찾을 키워드 : 458행 / 전체 674행중 맛차차
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=맛차차&sm=hty&style=v5
이번에 찾을 키워드 : 459행 / 전체 674행중 아이오쓰리 성수사거리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아이오쓰리%20성수사

이번에 찾을 키워드 : 516행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 517행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&sm=hty&style=v5
이번에 찾을 키워드 : 518행 / 전체 674행중 맛차차
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=맛차차&sm=hty&style=v5
이번에 찾을 키워드 : 519행 / 전체 674행중 아이오쓰리 성수사거리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아이오쓰리%20성수사거리점&sm=hty&style=v5
이번에 찾을 키워드 : 520행 / 전체 674행중 브라운팟
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=브라운팟&sm=hty&style=v5
이번에 찾을 키워드 : 521행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 522행 / 전체 674행중 카페노티드 피치스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20피치스&sm=hty&style=v5
이번에 찾을 키워드 : 523행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style

이번에 찾을 키워드 : 581행 / 전체 674행중 플디 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=플디%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 582행 / 전체 674행중 카페노티드 피치스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20피치스&sm=hty&style=v5
이번에 찾을 키워드 : 583행 / 전체 674행중 천상가옥
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=천상가옥&sm=hty&style=v5
이번에 찾을 키워드 : 584행 / 전체 674행중 비엔나커피하우스 서울숲역점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=비엔나커피하우스%20서울숲역점&sm=hty&style=v5
이번에 찾을 키워드 : 585행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 586행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 587행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 588행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/

이번에 찾을 키워드 : 645행 / 전체 674행중 훔볼트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=훔볼트&sm=hty&style=v5
이번에 찾을 키워드 : 646행 / 전체 674행중 업사이드커피 뚝섬점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=업사이드커피%20뚝섬점&sm=hty&style=v5
이번에 찾을 키워드 : 647행 / 전체 674행중 키커피컴퍼니 왕십리점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=키커피컴퍼니%20왕십리점&sm=hty&style=v5
이번에 찾을 키워드 : 648행 / 전체 674행중 카페온화 성수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페온화%20성수점&sm=hty&style=v5
이번에 찾을 키워드 : 649행 / 전체 674행중 오우드
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오우드&sm=hty&style=v5
이번에 찾을 키워드 : 650행 / 전체 674행중 카페노티드 성수
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20성수&sm=hty&style=v5
이번에 찾을 키워드 : 651행 / 전체 674행중 하루노유키
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=하루노유키&sm=hty&style=v5
이번에 찾을 키워드 : 652행 / 전체 674행중 마크69
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=마크69&

In [109]:
df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']

In [110]:
df['naver_map_url']

0      https://m.place.naver.com/restaurant/1725895520
1      https://m.place.naver.com/restaurant/1945220263
2        https://m.place.naver.com/restaurant/37879684
3      https://m.place.naver.com/restaurant/1925852247
4        https://m.place.naver.com/restaurant/37910590
                            ...                       
670    https://m.place.naver.com/restaurant/1165635472
671    https://m.place.naver.com/restaurant/1030413970
672    https://m.place.naver.com/restaurant/1022973489
673    https://m.place.naver.com/restaurant/1074358103
674    https://m.place.naver.com/restaurant/1320858854
Name: naver_map_url, Length: 675, dtype: object

In [111]:
df = df.loc[df['naver_map_url'] != 'https://m.place.naver.com/restaurant/']
df

,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,1492599844,블루보틀 성수 카페,서울 성동구 성수동1가 656-302,서울 성동구 아차산로 7,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/1492599844,블루보틀 성수 카페,https://m.place.naver.com/restaurant/1725895520
1,1,1301973155,할아버지공장,서울 성동구 성수동2가 309-133,서울 성동구 성수이로7가길 9,010-3934-2301,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1301973155,할아버지공장,https://m.place.naver.com/restaurant/1945220263
2,2,1783691580,쎈느,서울 성동구 성수동2가 314-2,서울 성동구 연무장5길 20,02-469-2122,카페,음식점 > 카페,http://place.map.kakao.com/1783691580,쎈느,https://m.place.naver.com/restaurant/37879684
3,3,145791269,어니언컴퍼니,서울 성동구 성수동2가 277-135,서울 성동구 아차산로9길 8,02-1644-1941,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/145791269,어니언컴퍼니,https://m.place.naver.com/restaurant/1925852247
4,4,1128989913,대림창고,서울 성동구 성수동2가 322-32,서울 성동구 성수이로 78,02-499-9669,카페,음식점 > 카페 > 테마카페 > 갤러리카페,http://place.map.kakao.com/1128989913,대림창고,https://m.place.naver.com/restaurant/37910590
...,...,...,...,...,...,...,...,...,...,...,...
670,670,753723493,브라운팟,서울 성동구 행당동 286-19,서울 성동구 무학봉28길 16,02-2299-7796,카페,음식점 > 카페,http://place.map.kakao.com/753723493,브라운팟,https://m.place.naver.com/restaurant/1165635472
671,671,1431120740,플디 성수점,서울 성동구 성수동1가 13-156,서울 성동구 상원길 44,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1431120740,플디 성수점,https://m.place.naver.com/restaurant/1030413970
672,672,1300323227,카페노티드 피치스,서울 성동구 성수동2가 301-16,서울 성동구 연무장3길 9,0507-1405-9377,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1300323227,카페노티드 피치스,https://m.place.naver.com/restaurant/1022973489
673,673,1501931765,천상가옥,서울 성동구 성수동2가 322-4,서울 성동구 성수이로14길 14,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1501931765,천상가옥,https://m.place.naver.com/restaurant/1074358103


In [112]:
df.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/성동구 카페 정보_url.csv')
df.to_json('/Users/dc/practice/recommendation/recomm_drf/data/crawl/cafe/성동구 카페 정보_url.json', orient='records', indent=4, force_ascii=False)

In [121]:
# 2022.10.26
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
import time
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import re
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [122]:
df = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe', '성동구 카페 정보_url.csv'), sep=',')
#df = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/성동구 카페 정보_url.csv'), sep=',')

# sample_test(2022.10.27)
#df = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/성동구 카페 정보_url_sample.csv'), sep=',')

In [123]:
df

,Unnamed: 0.1,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,0,1492599844,블루보틀 성수 카페,서울 성동구 성수동1가 656-302,서울 성동구 아차산로 7,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/1492599844,블루보틀 성수 카페,https://m.place.naver.com/restaurant/1725895520
1,1,1,1301973155,할아버지공장,서울 성동구 성수동2가 309-133,서울 성동구 성수이로7가길 9,010-3934-2301,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1301973155,할아버지공장,https://m.place.naver.com/restaurant/1945220263
2,2,2,1783691580,쎈느,서울 성동구 성수동2가 314-2,서울 성동구 연무장5길 20,02-469-2122,카페,음식점 > 카페,http://place.map.kakao.com/1783691580,쎈느,https://m.place.naver.com/restaurant/37879684
3,3,3,145791269,어니언컴퍼니,서울 성동구 성수동2가 277-135,서울 성동구 아차산로9길 8,02-1644-1941,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/145791269,어니언컴퍼니,https://m.place.naver.com/restaurant/1925852247
4,4,4,1128989913,대림창고,서울 성동구 성수동2가 322-32,서울 성동구 성수이로 78,02-499-9669,카페,음식점 > 카페 > 테마카페 > 갤러리카페,http://place.map.kakao.com/1128989913,대림창고,https://m.place.naver.com/restaurant/37910590
...,...,...,...,...,...,...,...,...,...,...,...,...
670,670,670,753723493,브라운팟,서울 성동구 행당동 286-19,서울 성동구 무학봉28길 16,02-2299-7796,카페,음식점 > 카페,http://place.map.kakao.com/753723493,브라운팟,https://m.place.naver.com/restaurant/1165635472
671,671,671,1431120740,플디 성수점,서울 성동구 성수동1가 13-156,서울 성동구 상원길 44,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1431120740,플디 성수점,https://m.place.naver.com/restaurant/1030413970
672,672,672,1300323227,카페노티드 피치스,서울 성동구 성수동2가 301-16,서울 성동구 연무장3길 9,0507-1405-9377,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1300323227,카페노티드 피치스,https://m.place.naver.com/restaurant/1022973489
673,673,673,1501931765,천상가옥,서울 성동구 성수동2가 322-4,서울 성동구 성수이로14길 14,NaN,카페,음식점 > 카페,http://place.map.kakao.com/1501931765,천상가옥,https://m.place.naver.com/restaurant/1074358103


In [124]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

# 네이버 지도의 유형 리스트
naver_map_type_list = []
# # 블로그 리뷰 리스트
# blog_review_list = []
# 블로그 리뷰 수 리스트
blog_review_qty_list = []
# 별점 점수(2022년 종료 예정) 정보 리스트
stars_list = []
# 별점 평가 수(2022년 종료 예정) 리스트
stars_qty_list = []
# 방문자 평가 참여자수 리스트
visitor_review_qty_list = []
# 방문자 리뷰 태그(호평) 리스트
visitor_review_list = ['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                       '양이 많아요', '혼밥하기 좋아요']


df2 = df[['id', 'place_name', 'address_name', 'phone']]
# df2 = pd.DataFrame(df, columns=['id', 'place_name', 'address_name', 'phone'])



for i in visitor_review_list:
    df2[i] = ''

print(df2)
# print('==============')
# df2.iloc[:, 7]




             id      place_name          address_name           phone  \
0    1492599844      블루보틀 성수 카페  서울 성동구 성수동1가 656-302       1533-6906   
1    1301973155          할아버지공장  서울 성동구 성수동2가 309-133   010-3934-2301   
2    1783691580              쎈느    서울 성동구 성수동2가 314-2     02-469-2122   
3     145791269          어니언컴퍼니  서울 성동구 성수동2가 277-135    02-1644-1941   
4    1128989913            대림창고   서울 성동구 성수동2가 322-32     02-499-9669   
..          ...             ...                   ...             ...   
670   753723493            브라운팟     서울 성동구 행당동 286-19    02-2299-7796   
671  1431120740          플디 성수점   서울 성동구 성수동1가 13-156             NaN   
672  1300323227       카페노티드 피치스   서울 성동구 성수동2가 301-16  0507-1405-9377   
673  1501931765            천상가옥    서울 성동구 성수동2가 322-4             NaN   
674   157479724  비엔나커피하우스 서울숲역점  서울 성동구 성수동1가 656-447   070-4628-5049   

    커피가 맛있어요 디저트가 맛있어요 빵이 맛있어요 친절해요 인테리어가 멋져요 매장이 청결해요  ... 화장실이 깨끗해요  \
0                                                 

/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_62661/1163794770.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i] = ''
/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_62661/1163794770.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i] = ''
/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_62661/1163794770.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [5]:
dict_review = {}
for i in visitor_review_list:
    dict_review[i] = []
    
print(dict_review)

# dict_review ={'커피가 맛있어요' : [], '빵이 맛있어요' : [], '친절해요' : []}

{'커피가 맛있어요': [], '디저트가 맛있어요': [], '빵이 맛있어요': [], '친절해요': [], '인테리어가 멋져요': [], '매장이 청결해요': [], '뷰가 좋아요': [], '음료가 맛있어요': [], '사진이 잘 나와요': [], '특별한 메뉴가 있어요': [], '대화하기 좋아요': [], '가성비가 좋아요': [], '집중하기 좋아요': [], '화장실이 깨끗해요': [], '주차하기 편해요': [], '좌석이 편해요': [], '음식이 맛있어요': [], '매장이 넓어요': [], '단체모임 하기 좋아요': [], '특별한 날 가기 좋아요': [], '재료가 신선해요': [], '양이 많아요': [], '혼밥하기 좋아요': []}


In [7]:
#chromedriver = '/Users/dc/practice/recommendation/recomm_drf/cafe/chromedriver' 
chromedriver = '/Users/dc/study_dir/recommendation/cafe/chromedriver'

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)


/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_62661/94347290.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)
/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_62661/94347290.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  sub_driver = webdriver.Chrome(chromedriver)


In [8]:
# bs4 이용

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):
    driver.get(url)
    sub_driver.get(url+"/review/visitor")
    time.sleep(3.0)
    
    html_driver = driver.page_source
    soup1 = BeautifulSoup(html_driver, 'html.parser')
    
    html_sub_driver = sub_driver.page_source
    soup2 = BeautifulSoup(html_sub_driver, 'html.parser')
    
    time.sleep(2.0)
    
    print('============='*8)
    print(f'{i+1}번째 데이터 수집 시작')
    
    # 네이버 지도의 유형 분류
    try: 
        naver_map_type = soup1.select_one(".DJJvD").text
        print(f'naver_map_type: {naver_map_type}')
        naver_map_type_list.append(naver_map_type)    
    except:
        print('Element 요소 미존재')
        naver_map_type_list.append('null')
        
    print(f'naver_map_type_list 길이: {len(naver_map_type_list)}')
        
    time.sleep(1.0)
        
    # 블로그 리뷰 수
    try:
        blog_review_qty = soup1.select(".place_bluelink>em")[1].text
        print(f'블로그 리뷰 수: {blog_review_qty}')
        blog_review_qty_list.append(blog_review_qty)
    except:
        print('Element 요소 미존재')
        blog_review_qty_list.append('null')
        
    print(f'blog_review_qty_list 길이: {len(blog_review_qty_list)}')
        
    time.sleep(1.0)
        
    # 별점 점수(2022년 종료 예정)
    try:
        star_review_stars = soup1.select_one("div.dAsGb>span.PXMot.LXIwF>em").text 
        print(f'별점 점수(2022년 종료 예정): {star_review_stars}')
        stars_list.append(star_review_stars)
    except:
        print('Element 요소 미존재')
        stars_list.append('null')
        
    print(f'stars_list 길이: {len(stars_list)}')
        
    time.sleep(0.5)
        
    # 별점 평가 수(2022년 종료 예정)
    try:
        star_review_qty = soup2.select("div.Xj_yJ>span.m7jAR")[1]
        star_review_qty = star_review_qty.text
        print(f'별점 평가 수(2022년 종료 예정): {star_review_qty}')
        stars_qty_list.append(star_review_qty)
    except:
        print('Element 요소 미존재')
        stars_qty_list.append('null')
        
    print(f'stars_qty_list 길이: {len(stars_qty_list)}')
        
    time.sleep(0.5)
        
    # 방문자 평가 참여자수 + # 방문자 호평 부분
    try: # 방문자 평가 참여자수
        visitor_review_qty = soup2.select_one("div._Wmab>span.BPjKc").text
        visitor_review_qty = re.sub(r'[()]', '', visitor_review_qty)
        visitor_review_qty = re.sub(r'명 참여', '', visitor_review_qty)
        print(f'방문자 평가 참여자수: {visitor_review_qty}')
        visitor_review_qty_list.append(visitor_review_qty)
        
        # 방문자 호평 부분
        exist_review = []

        visitor_eval_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.nWiXa")
        visitor_choice_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.TwM9q")

        for i in range(5):
            visitor_eval = visitor_eval_list[i].text
            text_evaluation = re.sub('["]', '', visitor_eval)
            #print(f'방문자 호평 부분: {text_evaluation}')
            visitor_choice = visitor_choice_list[i].text
            visitor_choice = re.sub('이 키워드를 선택한 인원', '', visitor_choice)
            visitor_choice = visitor_choice.strip()
            #print(f'방문자 호평수: {visitor_choice}')
            if text_evaluation in visitor_review_list:
                dict_review[text_evaluation].append(visitor_choice)
                exist_review.append(text_evaluation)
                print(f'해당 플레이스 리뷰: {exist_review}')
        for review in visitor_review_list:
            if review not in exist_review:
                dict_review[review].append('')
        
    except:
        # 방문자 평가 참여자수
        print(f'방문자 평가 참여자수 10명 미만인 장소')
        visitor_review_qty = '9'
        visitor_review_qty_list.append(visitor_review_qty)
        # 방문자 호평 부분
        for i in visitor_review_list:
            dict_review[i].append('')
        
#     # 방문자 호평 부분
#     try:
#         exist_review = []

#         visitor_eval_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.nWiXa")
#         visitor_choice_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.TwM9q")

#         for i in range(5):
#             visitor_eval = visitor_eval_list[i].text
#             text_evaluation = re.sub('["]', '', visitor_eval)
#             #print(f'방문자 호평 부분: {text_evaluation}')
#             visitor_choice = visitor_choice_list[i].text
#             visitor_choice = re.sub('이 키워드를 선택한 인원', '', visitor_choice)
#             visitor_choice = visitor_choice.strip()
#             #print(f'방문자 호평수: {visitor_choice}')
#             if text_evaluation in visitor_review_list:
#                 dict_review[text_evaluation].append(visitor_choice)
#                 exist_review.append(text_evaluation)
#                 print(f'해당 플레이스 리뷰: {exist_review}')
#         for review in visitor_review_list:
#             if review not in exist_review:
#                 dict_review[review].append('')
#     except:
#         for i in visitor_review_list:
#             dict_review[i].append('')


/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_62661/869230767.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):


  0%|          | 0/675 [00:00<?, ?it/s]

1번째 데이터 수집 시작
naver_map_type: 블루보틀
naver_map_type_list 길이: 1
블로그 리뷰 수: 3,525
blog_review_qty_list 길이: 1
별점 점수(2022년 종료 예정): 4.4
stars_list 길이: 1
별점 평가 수(2022년 종료 예정): 12,644개 평점 12,893명 참여
stars_qty_list 길이: 1
방문자 평가 참여자수: 3,607
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '인테리어가 멋져요', '매장이 청결해요']
2번째 데이터 수집 시작
naver_map_type: 원목가구
naver_map_type_list 길이: 2
블로그 리뷰 수: 6
blog_review_qty_list 길이: 2
Element 요소 미존재
stars_list 길이: 2
Element 요소 미존재
stars_qty_list 길이: 2
방문자 평가 참여자수 10명 미만인 장소
3번째 데이터 수집 시작
naver_map_type: 의류제작
naver_map_type_list 길이: 3
블로그 리뷰 수: 3
blog_review_qty_list 길이: 3
Element 요소 미존재
stars_list 길이: 3
Element 요소 미존재
stars_qty_list 길이: 3
방문자 평가 참여자수 10명 미만인 장소
4번째 데이터 수집 시작
naver_map_type: 기업
naver_map_type_list 길이: 4
Element 요소 미존재
blog_review_qty_list 길이: 4
Element 요소 미존재
stars_list 길이: 4
Element 요소 미존재
stars_qty_

17번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 17
블로그 리뷰 수: 808
blog_review_qty_list 길이: 17
별점 점수(2022년 종료 예정): 4.45
stars_list 길이: 17
별점 평가 수(2022년 종료 예정): 1,210개 평점 1,300명 참여
stars_qty_list 길이: 17
방문자 평가 참여자수: 477
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '인테리어가 멋져요', '특별한 메뉴가 있어요']
18번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 18
블로그 리뷰 수: 974
blog_review_qty_list 길이: 18
별점 점수(2022년 종료 예정): 4.48
stars_list 길이: 18
별점 평가 수(2022년 종료 예정): 575개 평점 596명 참여
stars_qty_list 길이: 18
방문자 평가 참여자수: 231
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '인테리어가 멋져요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '인테리어가 멋져요', '친절해요', '음료가 맛있어요']
19번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_

별점 점수(2022년 종료 예정): 4.39
stars_list 길이: 31
별점 평가 수(2022년 종료 예정): 1,090개 평점 1,241명 참여
stars_qty_list 길이: 31
방문자 평가 참여자수: 461
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요', '매장이 청결해요']
32번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 32
블로그 리뷰 수: 321
blog_review_qty_list 길이: 32
별점 점수(2022년 종료 예정): 4.54
stars_list 길이: 32
별점 평가 수(2022년 종료 예정): 202개 평점 237명 참여
stars_qty_list 길이: 32
방문자 평가 참여자수: 129
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요', '특별한 메뉴가 있어요']
33번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 33
블로그 리뷰 수: 472
blog_review_qty_list 길이: 33
별점 점수(2022년 종료 예정): 4.48
stars_list 길이: 33
별점 평가 수(2022년 종료 예

...

601번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 601
블로그 리뷰 수: 407
blog_review_qty_list 길이: 601
별점 점수(2022년 종료 예정): 4.39
stars_list 길이: 601
별점 평가 수(2022년 종료 예정): 1,090개 평점 1,241명 참여
stars_qty_list 길이: 601
방문자 평가 참여자수: 464
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요', '매장이 청결해요']
602번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 602
블로그 리뷰 수: 321
blog_review_qty_list 길이: 602
별점 점수(2022년 종료 예정): 4.54
stars_list 길이: 602
별점 평가 수(2022년 종료 예정): 202개 평점 237명 참여
stars_qty_list 길이: 602
방문자 평가 참여자수: 129
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요', '특별한 메뉴가 있어요']
603번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_l

616번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 616
블로그 리뷰 수: 407
blog_review_qty_list 길이: 616
별점 점수(2022년 종료 예정): 4.39
stars_list 길이: 616
별점 평가 수(2022년 종료 예정): 1,090개 평점 1,241명 참여
stars_qty_list 길이: 616
방문자 평가 참여자수: 464
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요', '매장이 청결해요']
617번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 617
블로그 리뷰 수: 321
blog_review_qty_list 길이: 617
별점 점수(2022년 종료 예정): 4.54
stars_list 길이: 617
별점 평가 수(2022년 종료 예정): 202개 평점 237명 참여
stars_qty_list 길이: 617
방문자 평가 참여자수: 129
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요', '특별한 메뉴가 있어요']
618번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_l

631번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 631
블로그 리뷰 수: 407
blog_review_qty_list 길이: 631
별점 점수(2022년 종료 예정): 4.39
stars_list 길이: 631
별점 평가 수(2022년 종료 예정): 1,090개 평점 1,241명 참여
stars_qty_list 길이: 631
방문자 평가 참여자수: 464
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요', '매장이 청결해요']
632번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 632
블로그 리뷰 수: 321
blog_review_qty_list 길이: 632
별점 점수(2022년 종료 예정): 4.54
stars_list 길이: 632
별점 평가 수(2022년 종료 예정): 202개 평점 237명 참여
stars_qty_list 길이: 632
방문자 평가 참여자수: 129
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요', '특별한 메뉴가 있어요']
633번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_l

646번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 646
블로그 리뷰 수: 407
blog_review_qty_list 길이: 646
별점 점수(2022년 종료 예정): 4.39
stars_list 길이: 646
별점 평가 수(2022년 종료 예정): 1,090개 평점 1,241명 참여
stars_qty_list 길이: 646
방문자 평가 참여자수: 464
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요', '매장이 청결해요']
647번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 647
블로그 리뷰 수: 321
blog_review_qty_list 길이: 647
별점 점수(2022년 종료 예정): 4.54
stars_list 길이: 647
별점 평가 수(2022년 종료 예정): 202개 평점 237명 참여
stars_qty_list 길이: 647
방문자 평가 참여자수: 129
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요', '특별한 메뉴가 있어요']
648번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_l

661번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 661
블로그 리뷰 수: 407
blog_review_qty_list 길이: 661
별점 점수(2022년 종료 예정): 4.39
stars_list 길이: 661
별점 평가 수(2022년 종료 예정): 1,090개 평점 1,241명 참여
stars_qty_list 길이: 661
방문자 평가 참여자수: 464
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '인테리어가 멋져요', '음료가 맛있어요', '매장이 청결해요']
662번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 662
블로그 리뷰 수: 321
blog_review_qty_list 길이: 662
별점 점수(2022년 종료 예정): 4.54
stars_list 길이: 662
별점 평가 수(2022년 종료 예정): 202개 평점 237명 참여
stars_qty_list 길이: 662
방문자 평가 참여자수: 129
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요', '특별한 메뉴가 있어요']
663번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_l

In [9]:
driver.quit()
sub_driver.quit()

In [10]:
print(dict_review)

{'커피가 맛있어요': ['2336', '', '', '', '313', '531', '193', '923', '524', '179', '89', '367', '281', '111', '276', '106', '308', '97', '75', '77', '191', '146', '', '231', '529', '', '58', '52', '68', '95', '238', '117', '92', '31', '262', '249', '47', '113', '', '65', '191', '26', '', '', '144', '238', '117', '92', '31', '262', '249', '47', '113', '', '65', '191', '26', '', '', '144', '238', '117', '92', '31', '262', '249', '47', '113', '', '65', '191', '26', '', '', '144', '238', '117', '92', '31', '262', '249', '47', '113', '', '65', '191', '26', '', '', '144', '238', '117', '92', '31', '262', '249', '47', '113', '', '65', '191', '26', '', '', '144', '238', '117', '92', '31', '262', '249', '47', '113', '', '65', '191', '26', '', '', '144', '238', '117', '92', '31', '262', '249', '47', '113', '', '65', '190', '26', '', '', '144', '238', '117', '92', '31', '262', '249', '47', '113', '', '65', '190', '26', '', '', '144', '238', '117', '92', '31', '262', '249', '47', '113', '', '65', '190', 

In [11]:
print(len(dict_review['혼밥하기 좋아요']))

675


In [12]:
print(len(naver_map_type_list))

675


In [13]:
df2['naver_map_type'] = naver_map_type_list            # 네이버 지도의 유형 리스트
df2['blog_review_qty_list'] = blog_review_qty_list     # 블로그 리뷰 수 리스트
df2['stars'] = stars_list                              # 별점 점수(2022년 종료 예정) 정보 리스트
df2['stars_qty'] = stars_qty_list                      # 별점 평가 수(2022년 종료 예정) 리스트
df2['visitor_review_qty'] = visitor_review_qty_list    # 방문자 평가 참여자수 리스트 


for key in dict_review:
    df2[key] = dict_review[key]

In [14]:
print(df2)

             id      place_name          address_name           phone  \
0    1492599844      블루보틀 성수 카페  서울 성동구 성수동1가 656-302       1533-6906   
1    1301973155          할아버지공장  서울 성동구 성수동2가 309-133   010-3934-2301   
2    1783691580              쎈느    서울 성동구 성수동2가 314-2     02-469-2122   
3     145791269          어니언컴퍼니  서울 성동구 성수동2가 277-135    02-1644-1941   
4    1128989913            대림창고   서울 성동구 성수동2가 322-32     02-499-9669   
..          ...             ...                   ...             ...   
670   753723493            브라운팟     서울 성동구 행당동 286-19    02-2299-7796   
671  1431120740          플디 성수점   서울 성동구 성수동1가 13-156             NaN   
672  1300323227       카페노티드 피치스   서울 성동구 성수동2가 301-16  0507-1405-9377   
673  1501931765            천상가옥    서울 성동구 성수동2가 322-4             NaN   
674   157479724  비엔나커피하우스 서울숲역점  서울 성동구 성수동1가 656-447   070-4628-5049   

    커피가 맛있어요 디저트가 맛있어요 빵이 맛있어요 친절해요 인테리어가 멋져요 매장이 청결해요  ... 단체모임 하기 좋아요  \
0       2336                    968       733   

In [15]:
#df2 내용 csv로 저장
#df2.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/sample.csv')
df2.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/성동구_카페_review.csv', mode='w')

In [17]:
df2_json = df2.to_json('/Users/dc/study_dir/recommendation/data/crawl/cafe/성동구_카페_review.json', orient='records', indent=4, force_ascii=False)
#df2_json = df2.to_json('/Users/dc/practice/recommendation/recomm_drf/data/crawl/cafe/성동구_review.json', orient='records', indent=4, force_ascii=False)
# print(df2_json)

In [125]:
# test(2022.10.25(화))
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
import time
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import re
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [126]:
#df3 = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe', '성동구_review_rating.csv'), sep=',')
df3 = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe', '성동구_카페_rating.csv'), sep=',')
df3 = df3.iloc[:, 1:4]
df3

,place_name,review,rating
0,블루보틀 성수 카페,커피가 맛있어요,2336.0
1,블루보틀 성수 카페,디저트가 맛있어요,NaN
2,블루보틀 성수 카페,빵이 맛있어요,NaN
3,블루보틀 성수 카페,친절해요,968.0
4,블루보틀 성수 카페,인테리어가 멋져요,733.0
...,...,...,...
15520,비엔나커피하우스 서울숲역점,단체모임 하기 좋아요,NaN
15521,비엔나커피하우스 서울숲역점,특별한 날 가기 좋아요,NaN
15522,비엔나커피하우스 서울숲역점,재료가 신선해요,NaN
15523,비엔나커피하우스 서울숲역점,양이 많아요,NaN


In [127]:
df4 = df3.dropna()
df4

,place_name,review,rating
0,블루보틀 성수 카페,커피가 맛있어요,2336.0
3,블루보틀 성수 카페,친절해요,968.0
4,블루보틀 성수 카페,인테리어가 멋져요,733.0
5,블루보틀 성수 카페,매장이 청결해요,692.0
7,블루보틀 성수 카페,음료가 맛있어요,805.0
...,...,...,...
15502,비엔나커피하우스 서울숲역점,커피가 맛있어요,144.0
15503,비엔나커피하우스 서울숲역점,디저트가 맛있어요,57.0
15505,비엔나커피하우스 서울숲역점,친절해요,49.0
15509,비엔나커피하우스 서울숲역점,음료가 맛있어요,66.0


In [128]:
few_reviews = []
for i in visitor_review_list:
    review_cnt = len(df4.loc[df4['review'] == i])
    print(f'{i} 리뷰 조회 갯수: {review_cnt}')
    if review_cnt < 2:
        few_reviews.append(i)

커피가 맛있어요 리뷰 조회 갯수: 541
디저트가 맛있어요 리뷰 조회 갯수: 450
빵이 맛있어요 리뷰 조회 갯수: 43
친절해요 리뷰 조회 갯수: 403
인테리어가 멋져요 리뷰 조회 갯수: 325
매장이 청결해요 리뷰 조회 갯수: 306
뷰가 좋아요 리뷰 조회 갯수: 46
음료가 맛있어요 리뷰 조회 갯수: 538
사진이 잘 나와요 리뷰 조회 갯수: 48
특별한 메뉴가 있어요 리뷰 조회 갯수: 310
대화하기 좋아요 리뷰 조회 갯수: 90
가성비가 좋아요 리뷰 조회 갯수: 1
집중하기 좋아요 리뷰 조회 갯수: 0
화장실이 깨끗해요 리뷰 조회 갯수: 0
주차하기 편해요 리뷰 조회 갯수: 0
좌석이 편해요 리뷰 조회 갯수: 43
음식이 맛있어요 리뷰 조회 갯수: 1
매장이 넓어요 리뷰 조회 갯수: 0
단체모임 하기 좋아요 리뷰 조회 갯수: 0
특별한 날 가기 좋아요 리뷰 조회 갯수: 0
재료가 신선해요 리뷰 조회 갯수: 0
양이 많아요 리뷰 조회 갯수: 0
혼밥하기 좋아요 리뷰 조회 갯수: 0


In [129]:
print(few_reviews)

['가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요', '양이 많아요', '혼밥하기 좋아요']


In [130]:
df5 = df4.copy()
for i in few_reviews:
    df5 = df5[df5['review']!= i]
#     idx = df5[df5['review'] == i].index
#     df5.drop(idx, inplace=True)
    
df5

,place_name,review,rating
0,블루보틀 성수 카페,커피가 맛있어요,2336.0
3,블루보틀 성수 카페,친절해요,968.0
4,블루보틀 성수 카페,인테리어가 멋져요,733.0
5,블루보틀 성수 카페,매장이 청결해요,692.0
7,블루보틀 성수 카페,음료가 맛있어요,805.0
...,...,...,...
15502,비엔나커피하우스 서울숲역점,커피가 맛있어요,144.0
15503,비엔나커피하우스 서울숲역점,디저트가 맛있어요,57.0
15505,비엔나커피하우스 서울숲역점,친절해요,49.0
15509,비엔나커피하우스 서울숲역점,음료가 맛있어요,66.0


In [131]:
dup = df5.duplicated(['review', 'place_name'])
dup

0        False
3        False
4        False
5        False
7        False
         ...  
15502     True
15503     True
15505     True
15509     True
15512     True
Length: 3143, dtype: bool

In [132]:
df6 = pd.concat([df5, dup], axis=1)
df6.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/성동구 카페 정보_전처리3.csv')

In [90]:
df5.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/성동구 카페 정보_전처리1.csv')

In [104]:
#df5 = df5.drop_duplicates()
#df5

# df6 = df5.groupby(['review', 'place_name'], as_index=False).mean()
# df6.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/성동구 카페 정보_전처리2.csv')

In [73]:
x = df5.copy()
y = df5['review']

# 테스트 샘플 데이터에서 사용
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# 실제 대용량 데이터에서 사용
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [64]:
x_train

,place_name,review,rating
5962,오우드,매장이 청결해요,118.0
10909,아이오쓰리 성수사거리점,음료가 맛있어요,64.0
3174,카페온화 성수점,커피가 맛있어요,31.0
15065,브라운팟,커피가 맛있어요,190.0
12544,카페노티드 성수,특별한 메뉴가 있어요,507.0
...,...,...,...
1151,카페노티드 성수,디저트가 맛있어요,1978.0
14290,하루노유키,음료가 맛있어요,48.0
1042,훔볼트,음료가 맛있어요,91.0
11124,카페온화 성수점,좌석이 편해요,18.0


In [49]:
#x_train = x_train.groupby(['review', 'place_name'])

In [50]:
#x_train.shape

In [75]:
rating_matrix = pd.pivot_table(x_train, index='review', columns='place_name', values = 'rating', aggfunc='mean')
rating_matrix

place_name,5TO7,갈십리,글로우,대림창고,도렐 육지3호점,로우커피스탠드,로우키,마를리,마크69,맛차차,...,카페노티드 성수,카페노티드 피치스,카페오롯,카페온화 성수점,카페포제,키커피컴퍼니 왕십리점,플디 성수점,하루&원데이,하루노유키,훔볼트
review,,,,,,,,,,,,,,,,,,,,,
대화하기 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,46.000000,NaN,...,NaN,NaN,32.0,NaN,NaN,NaN,NaN,100.0,NaN,NaN
디저트가 맛있어요,239.0,50.0,NaN,140.0,131.0,NaN,NaN,453.0,49.000000,NaN,...,1978.00000,954.000000,32.0,42.0,32.0,99.000000,68.0,NaN,NaN,250.781250
매장이 청결해요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,334.00000,NaN,NaN,23.0,33.0,43.722222,NaN,134.0,NaN,71.000000
뷰가 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
빵이 맛있어요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.000000,NaN
사진이 잘 나와요,NaN,19.0,NaN,90.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,244.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
음료가 맛있어요,121.0,39.0,71.0,NaN,127.0,77.0,43.0,243.0,52.733333,40.000000,...,NaN,NaN,30.0,20.0,NaN,39.000000,22.0,NaN,48.000000,91.000000
인테리어가 멋져요,114.0,55.0,50.0,264.0,108.0,18.0,NaN,288.0,103.766667,24.000000,...,NaN,480.000000,40.0,NaN,50.0,41.657143,NaN,NaN,NaN,127.586207
좌석이 편해요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
rating_matrix = x_train.pivot(index='review', columns='place_name', values='rating')
rating_matrix

ValueError: Index contains duplicate entries, cannot reshape

In [72]:
matrix_dummy = rating_matrix.copy().fillna(0)
review_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
review_similarity = pd.DataFrame(review_similarity,
                               index=rating_matrix.index,
                               columns=rating_matrix.index)

In [73]:
review_similarity

review,가성비가 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,음료가 맛있어요,음식이 맛있어요,인테리어가 멋져요,좌석이 편해요,친절해요,커피가 맛있어요,특별한 메뉴가 있어요
review,,,,,,,,,,,,,,
가성비가 좋아요,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110173,0.000000,0.000000,0.000000,0.059961,0.192948,0.000000
대화하기 좋아요,0.000000,1.000000,0.004658,0.000000,0.000000,0.000000,0.000000,0.326976,0.000000,0.025663,0.000000,0.000000,0.030721,0.000000
디저트가 맛있어요,0.000000,0.004658,1.000000,0.020546,0.009505,0.000000,0.046031,0.265068,0.000000,0.511669,0.014306,0.498299,0.662957,0.669621
매장이 청결해요,0.000000,0.000000,0.020546,1.000000,0.000000,0.000000,0.000000,0.012019,0.186507,0.663888,0.032012,0.751005,0.039960,0.036137
뷰가 좋아요,0.000000,0.000000,0.009505,0.000000,1.000000,0.000000,0.174752,0.218543,0.000000,0.130228,0.000000,0.094586,0.021531,0.038136
빵이 맛있어요,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.068679,0.000000,0.000000,0.000000,0.043172,0.000000,0.072251
사진이 잘 나와요,0.000000,0.000000,0.046031,0.000000,0.174752,0.000000,1.000000,0.043692,0.000000,0.047292,0.000000,0.000000,0.253870,0.000000
음료가 맛있어요,0.110173,0.326976,0.265068,0.012019,0.218543,0.068679,0.043692,1.000000,0.000000,0.384129,0.000000,0.176928,0.651839,0.365135
음식이 맛있어요,0.000000,0.000000,0.000000,0.186507,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.188785


In [74]:
#### 플레이스 전반에 대한 review 경향을 고려한 함수 ####. axis=1 은 행 기준으로 평균을 구함(여기서 행은 review)
rating_mean = rating_matrix.mean(axis=1)
rating_mean

review
가성비가 좋아요       146.000000
대화하기 좋아요        77.666667
디저트가 맛있어요      356.105263
매장이 청결해요       155.000000
뷰가 좋아요         111.500000
빵이 맛있어요         98.000000
사진이 잘 나와요       59.000000
음료가 맛있어요       108.227273
음식이 맛있어요       346.000000
인테리어가 멋져요      165.263158
좌석이 편해요         18.000000
친절해요           172.222222
커피가 맛있어요       210.000000
특별한 메뉴가 있어요    149.500000
dtype: float64

In [75]:
# 각 플레이스에 대한 리뷰 전체적인 평점
rating_matrix.T

review,가성비가 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,음료가 맛있어요,음식이 맛있어요,인테리어가 멋져요,좌석이 편해요,친절해요,커피가 맛있어요,특별한 메뉴가 있어요
place_name,,,,,,,,,,,,,,
5TO7,NaN,NaN,239.0,NaN,NaN,NaN,NaN,121.0,NaN,NaN,NaN,NaN,231.0,NaN
갈십리,NaN,NaN,NaN,NaN,NaN,NaN,19.0,39.0,NaN,55.0,NaN,NaN,75.0,NaN
글로우,NaN,NaN,192.0,NaN,NaN,NaN,NaN,71.0,NaN,50.0,NaN,68.0,NaN,NaN
대림창고,NaN,NaN,140.0,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,313.0,NaN
도렐 육지3호점,NaN,NaN,131.0,NaN,NaN,NaN,NaN,127.0,NaN,108.0,NaN,NaN,308.0,101.0
로우커피스탠드,146.0,NaN,NaN,NaN,NaN,NaN,NaN,77.0,NaN,NaN,NaN,75.0,281.0,NaN
로우키,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,NaN,64.0,179.0,NaN
마를리,NaN,155.0,NaN,NaN,NaN,NaN,NaN,243.0,NaN,NaN,NaN,NaN,NaN,NaN
마크69,NaN,46.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,103.0,NaN,NaN,113.0,NaN


In [76]:
# 각 플레이스 평점(rating_matrix.T)에 대한 리뷰별 경향(rating_mean. 리뷰의 평점평균)을 고려하여 플레이스 평점평균 '편차(bias)' 계산
rating_matrix.T - rating_mean

review,가성비가 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,음료가 맛있어요,음식이 맛있어요,인테리어가 멋져요,좌석이 편해요,친절해요,커피가 맛있어요,특별한 메뉴가 있어요
place_name,,,,,,,,,,,,,,
5TO7,NaN,NaN,-117.105263,NaN,NaN,NaN,NaN,12.772727,NaN,NaN,NaN,NaN,21.0,NaN
갈십리,NaN,NaN,NaN,NaN,NaN,NaN,-40.0,-69.227273,NaN,-110.263158,NaN,NaN,-135.0,NaN
글로우,NaN,NaN,-164.105263,NaN,NaN,NaN,NaN,-37.227273,NaN,-115.263158,NaN,-104.222222,NaN,NaN
대림창고,NaN,NaN,-216.105263,NaN,NaN,NaN,31.0,NaN,NaN,NaN,NaN,NaN,103.0,NaN
도렐 육지3호점,NaN,NaN,-225.105263,NaN,NaN,NaN,NaN,18.772727,NaN,-57.263158,NaN,NaN,98.0,-48.5
로우커피스탠드,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-31.227273,NaN,NaN,NaN,-97.222222,71.0,NaN
로우키,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-101.263158,NaN,-108.222222,-31.0,NaN
마를리,NaN,77.333333,NaN,NaN,NaN,NaN,NaN,134.772727,NaN,NaN,NaN,NaN,NaN,NaN
마크69,NaN,-31.666667,-307.105263,NaN,NaN,NaN,NaN,NaN,NaN,-62.263158,NaN,NaN,-97.0,NaN


In [77]:
rating_bias = (rating_matrix.T - rating_mean).T
rating_bias

place_name,5TO7,갈십리,글로우,대림창고,도렐 육지3호점,로우커피스탠드,로우키,마를리,마크69,맛차차,...,카멜 성수점,카페 연무장,카페노티드 성수,카페오롯,카페온화 성수점,카페포제,키커피컴퍼니 왕십리점,하루&원데이,하루노유키,훔볼트
review,,,,,,,,,,,,,,,,,,,,,
가성비가 좋아요,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
대화하기 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.333333,-31.666667,NaN,...,NaN,NaN,NaN,-45.666667,NaN,NaN,NaN,NaN,NaN,NaN
디저트가 맛있어요,-117.105263,NaN,-164.105263,-216.105263,-225.105263,NaN,NaN,NaN,-307.105263,NaN,...,NaN,-279.105263,1621.894737,NaN,-314.105263,-324.105263,-257.105263,NaN,NaN,NaN
매장이 청결해요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-132.000000,NaN,-112.000000,-21.0,NaN,NaN
뷰가 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-80.500000,...,NaN,-13.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
빵이 맛있어요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
사진이 잘 나와요,NaN,-40.000000,NaN,31.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,6.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
음료가 맛있어요,12.772727,-69.227273,-37.227273,NaN,18.772727,-31.227273,NaN,134.772727,NaN,-68.227273,...,-59.227273,NaN,NaN,NaN,NaN,-74.227273,-69.227273,NaN,-60.227273,NaN
음식이 맛있어요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [78]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [79]:
# 모델별 RMSE를 계산하는 함수(test set 활용하여 계산)
def score(model, neigbor_size=0):
    # 테스트 데이터의 review와 place_name 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만듬
    id_pairs = zip(x_test['review'], x_test['place_name'])
    # 모든 review-플레이스 짝에 대해서 주어진 예측모델에 의해 예측값 계산 및 리스트형 데이터 생성
    y_pred = np.array([model(review, place, neigbor_size) for (review, place) in id_pairs])
    # 실제 평점값
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [34]:
# id_pairs_list = list(zip(x_test['review'], x_test['place_name']))
# print(id_pairs_list)

[('특별한 날 가기 좋아요', '카페키이로'), ('디저트가 맛있어요', '스태픽스'), ('단체모임 하기 좋아요', '내자상회'), ('주차하기 편해요', '어니언 안국점'), ('단체모임 하기 좋아요', '서울서 둘째로잘하는집'), ('친절해요', '내자상회'), ('음료가 맛있어요', '토즈모임센터 종로점'), ('가성비가 좋아요', '개뿔'), ('화장실이 깨끗해요', '카페 레이어드 안국점'), ('재료가 신선해요', '테라로사 광화문점'), ('친절해요', '산모퉁이'), ('재료가 신선해요', '라카페갤러리'), ('디저트가 맛있어요', '대충유원지 인왕산'), ('대화하기 좋아요', '카페노티드 안국'), ('뷰가 좋아요', '개뿔'), ('화장실이 깨끗해요', '카페노티드 안국'), ('집중하기 좋아요', '차마시는 뜰'), ('인테리어가 멋져요', '프릳츠 원서점'), ('화장실이 깨끗해요', '토즈모임센터 광화문점'), ('뷰가 좋아요', '도트블랭킷'), ('특별한 메뉴가 있어요', '개뿔'), ('사진이 잘 나와요', '산모퉁이'), ('친절해요', '슬로우파크'), ('디저트가 맛있어요', '슬로우파크'), ('특별한 날 가기 좋아요', '라카페갤러리'), ('양이 많아요', '토즈모임센터 광화문점'), ('주차하기 편해요', '카페키이로'), ('화장실이 깨끗해요', '대충유원지 인왕산'), ('음료가 맛있어요', 'H라운지'), ('집중하기 좋아요', '카페아르크'), ('특별한 메뉴가 있어요', '카페 하이웨이스트 익선점'), ('혼밥하기 좋아요', '개뿔'), ('커피가 맛있어요', '어니언 안국점'), ('음식이 맛있어요', '안녕인사동'), ('빵이 맛있어요', '청수당'), ('집중하기 좋아요', '카페노티드 안국'), ('음료가 맛있어요', '도트블랭킷'), ('빵이 맛있어요', '아키비스트'), ('주차하기 편해요', '서울서 둘째로잘하는집'), ('매장이 넓어요', '아키비스트'), ('음식이 맛있어요', '카페노티

In [80]:
# rating_matrix > 0 을 통해 나온 True 값들을 numpy array로 만들고, 이렇게 나온 True, False 구분값들을 1과 0의 binary 값으로 변환
rating_binary_1 = np.array(rating_matrix > 0).astype(float)
rating_binary_2 = rating_binary_1.T

counts = np.dot(rating_binary_1, rating_binary_2)
counts = pd.DataFrame(counts,
                      index=rating_matrix.index,
                      columns=rating_matrix.index).fillna(0)

In [81]:
# 각 review 평가가 되어있는 플레이스의 갯수를 나타내기 (대각선 = 각 review 평가가 되어있는 플레이스의 갯수, 다른 값들은 review간 공통으로 평가되어있는 플레이스의 갯수)
counts

review,가성비가 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,음료가 맛있어요,음식이 맛있어요,인테리어가 멋져요,좌석이 편해요,친절해요,커피가 맛있어요,특별한 메뉴가 있어요
review,,,,,,,,,,,,,,
가성비가 좋아요,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
대화하기 좋아요,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0
디저트가 맛있어요,0.0,1.0,19.0,3.0,1.0,0.0,2.0,9.0,0.0,12.0,1.0,8.0,15.0,5.0
매장이 청결해요,0.0,0.0,3.0,7.0,0.0,0.0,0.0,3.0,1.0,4.0,1.0,3.0,5.0,2.0
뷰가 좋아요,0.0,0.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,0.0,1.0,1.0,2.0
빵이 맛있어요,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
사진이 잘 나와요,0.0,0.0,2.0,0.0,1.0,0.0,4.0,2.0,0.0,2.0,0.0,0.0,3.0,0.0
음료가 맛있어요,1.0,1.0,9.0,3.0,3.0,1.0,2.0,22.0,0.0,10.0,0.0,11.0,16.0,8.0
음식이 맛있어요,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [82]:
SIG_LEVEL = 2
MIN_RATINGS = 1

In [83]:
def CF_knn_bias_sig(review, place, neighbor_size=0):
    if place in rating_bias.columns:
        # input으로 들어온 review와 다른 review 간의 유사도 추출
        sim_scores = review_similarity[review].copy()
        # 해당 플레이스에 대한 review 평가 경향고려한(rating_bias 데이터프레임) 평점 정보 추출
        place_ratings = rating_bias[place].copy()
        # 플레이스 평가에 등록되어있지 않은 review인지 추출
        no_rating = place_ratings.isnull() # True, False
        # review가 공통으로 등록되어있는 플레이스의 갯수를 할당
        common_counts = counts[review]
        
        low_significance = common_counts < SIG_LEVEL   # SIG_LEVEL = 미리 설정한 숫자 보다 작은 review는 True로 변환
        # 플레이스 평가등록되지 않은 review or 지정한 SIG_LEVEL보다 적게 공통의 플레이스를 평가한 review의 index 추출
        none_rating_idx = place_ratings[no_rating | low_significance].index

        place_ratings = place_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)

        if neighbor_size == 0:
            # review 평가 경향을 반영한 가중평균 (플레이스점수*가중치(review간 유사도) / 가중치의 합)
            prediction = np.dot(sim_scores, place_ratings) / sim_scores.sum()
            # '가중평균(위의 prediction)' + '플레이스 전반에 대한 해당 review 평균 평점' 하여 reveiw 카테고리의 최종 플레이스 예상 평점 구함 
            prediction += rating_mean[review]
        else: 
            if len(sim_scores) > MIN_RATINGS:     # MIN_RATINGS = 미리 설정한 최소 review보다 현재 플레이스를 평가한 유사한 review가 큰 경우만
                print('=============='*5)
                print('MIN_RATINGS 보다 큰 상황')
                print('=============='*5)
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                place_ratings = np.array(place_ratings)
                # 유사도 오름차순 정렬후 user index 추출
                review_idx = np.argsort(sim_scores)
                # 입력한 neighbor_size 만큼 높은 유사도 정보 추출
                sim_scores = sim_scores[review_idx][-neighbor_size:]
                # 평점 정보 추출
                place_ratings = place_ratings[review_idx][-neighbor_size:]
                # review 평가 경향을 반영한 가중평균 (플레이스점수*가중치(review간 유사도) / 가중치의 합)
                prediction = np.dot(sim_scores, place_ratings) / sim_scores.sum()
                # '가중평균(위의 prediction)' + '플레이스 전반에 대한 해당 review 평균 평점' 하여 reveiw 카테고리의 최종 플레이스 예상 평점 구함 
                prediction += rating_mean[review]
            else:
                print('=============='*5)
                print('MIN_RATINGS 보다 작은 상황')
                print('=============='*5)
                # 원래 review의 평균 평점을 예상값으로 지정
                prediction = rating_mean[review]
    else:   # place에 대항하는 플레이스 정보가 rating_bias에 없는 경우
        # 원래 review의 평균 평점을 예상값으로 지정
        prediction = rating_mean[review]  # 강의 원본

    return prediction

In [85]:
CF_knn_bias_sig('뷰가 좋아요', '센터커피 서울숲점', 3)

MIN_RATINGS 보다 큰 상황


205.7739062932504

In [89]:
# SIG_LEVEL = 3
# MIN_RATINGS = 3   # 1이상
score(CF_knn_bias_sig, neigbor_size=2)

MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황


253.40091762718515

In [75]:
# 원본
def recom_place(review, n_items, neighbor_size=0):
    # review가 등록되어 있는 플레이스 정보 추출
    review_place = rating_matrix.loc[review].copy()
    print(f'review_place: {review_place}')
    print('=============='*5)
    
    for place in rating_matrix.columns:
        # 해당 플레이스에 대한 review가 이미 존재하는 경우, 추천 리스트에서 제외???(장소 추천시스템에선 불필요...?)
        if pd.notnull(review_place.loc[place]):
            review_place.loc[place] = 0
        else:
            review_place.loc[place] = CF_knn_bias_sig(review, place, neighbor_size)
            
    place_sort = review_place.sort_values(ascending=False)[:n_items]
    print(f'place_sort: {place_sort}')
    recom_places = movies.loc[place_sort.index]
    recommendations = recom_places['title']
    return recommendations

In [90]:
# 테스트 중
def recom_place(review, n_items, neighbor_size=0):
    # review가 등록되어 있는 플레이스 정보 추출
    review_place = rating_matrix.loc[review].copy()
    print(f'review_place: {review_place}')
    print('=============='*5)
    
    for place in rating_matrix.columns:
        review_place.loc[place] = CF_knn_bias_sig(review, place, neighbor_size)
        
    print(f'review_place 최종: {review_place}')
            
    place_sort = review_place.sort_values(ascending=False)[:n_items]
    print(f'place_sort: {place_sort}')
#     recom_places = movies.loc[place_sort.index]
#     recommendations = recom_places['title']
#     return recommendations

In [92]:
recom_place('대화하기 좋아요', 3, 3)

review_place: place_name
5TO7              NaN
갈십리               NaN
글로우               NaN
대림창고              NaN
도렐 육지3호점          NaN
로우커피스탠드           NaN
로우키               NaN
마를리             155.0
마크69             46.0
맛차차               NaN
메쉬커피              NaN
멜로워 성수플래그쉽        NaN
봉순이네다락방           NaN
블루보틀 성수 카페        NaN
서울앵무새             NaN
센터커피 서울숲점         NaN
슈퍼말차 성수           NaN
아우프글렛 금호점         NaN
아이오쓰리 성수사거리점      NaN
어라운드데이            NaN
업사이드커피 뚝섬점        NaN
오우드               NaN
옹근달               NaN
우디집               NaN
인더매스 마장           NaN
체다앤올리             NaN
치카치카              NaN
카멜 성수점            NaN
카페 연무장            NaN
카페노티드 성수          NaN
카페오롯             32.0
카페온화 성수점          NaN
카페포제              NaN
키커피컴퍼니 왕십리점       NaN
하루&원데이            NaN
하루노유키             NaN
훔볼트               NaN
Name: 대화하기 좋아요, dtype: float64
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_

In [7]:
# ratings = df3.groupby(['place_name'])[['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
#             a                           '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
#                                        '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
#                                        '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
#                                        '양이 많아요', '혼밥하기 좋아요']].sum()

# ratings
ratings = df3.pivot(index=['place_name'],
         columns=['place_name'],
         values=['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                 '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                 '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                 '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                 '양이 많아요', '혼밥하기 좋아요'])

ratings

ValueError: The name place_name occurs multiple times, use a level number

In [161]:
df3 = df2[['place_name', '커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요'
                       '양이 많아요', '혼밥하기 좋아요']]

In [162]:
place_rating = df3.copy()
place_rating = place_rating.replace('', np.nan)
place_rating = place_rating.fillna(0)
# place_rating = place_rating.T
print(place_rating)
place_rating.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/sample2.csv')

       place_name 커피가 맛있어요 디저트가 맛있어요 빵이 맛있어요 친절해요 인테리어가 멋져요 매장이 청결해요 뷰가 좋아요  \
0      토즈모임센터 종로점        0         0       0    0         0        0      0   
1        카페노티드 안국        0      3102       0  871       333      423      0   
2         어니언 안국점     1396      2231       0  983      1436        0      0   
3      블루보틀 삼청 카페     1323         0       0  536       434      428    430   
4            스태픽스      230       166       0    0       148        0    323   
5           아키비스트     1618       437       0    0       356        0      0   
6           안녕인사동        0         0       0    0         0        0      0   
7            산모퉁이      144         0       0    0        72        0    324   
8         포비 광화문점      784         0    1580  509         0      265      0   
9          차마시는 뜰        0         0       0  219       164        0    192   
10      테라로사 광화문점      513       123       0    0       192      124      0   
11        프릳츠 원서점      712       455       0    0   

In [139]:
# rating = place_rating.groupby(['place_name'])[['커피가 맛있어요']].sum()
# rating = place_rating.groupby(['place_name'])[['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
#                        '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
#                        '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
#                        '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요'
#                        '양이 많아요', '혼밥하기 좋아요']].sum()


# rating

In [163]:
print(place_rating.columns[1:])
review_list = place_rating.columns[1:]
print(len(review_list))

Index(['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요',
       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요',
       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', '음식이 맛있어요',
       '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요양이 많아요', '혼밥하기 좋아요'],
      dtype='object')
22


In [164]:
rating_matrix = place_rating.pivot_table(index=['place_name'], 
                                   columns=['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                                       '양이 많아요', '혼밥하기 좋아요'], 
                                   values=['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                                       '양이 많아요', '혼밥하기 좋아요'],
                                  aggfunc='sum')


In [165]:
rating_matrix

place_name
H라운지
개뿔
나무사이로
내자상회
대충유원지 인왕산
도트블랭킷
라카페갤러리
부빙
블루보틀 삼청 카페
산모퉁이


In [159]:
rating_matrix = rating_matrix.stack()

ValueError: Must pass non-zero number of levels/codes

In [150]:
rating_matrix

place_name
H라운지
개뿔
나무사이로
내자상회
대충유원지 인왕산
도트블랭킷
라카페갤러리
부빙
블루보틀 삼청 카페
산모퉁이


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [86]:
x = place_rating.copy()
y = place_rating.iloc[0].to_list()
print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

['토즈모임센터 종로점', '카페노티드 안국', '어니언 안국점', '블루보틀 삼청 카페', '스태픽스', '아키비스트', '안녕인사동', '산모퉁이', '포비 광화문점', '차마시는 뜰', '테라로사 광화문점', '프릳츠 원서점', '카페 레이어드 안국점', '나무사이로', '포비브라이트 광화문점', 'H라운지', '토즈모임센터 광화문점', '개뿔', '청수당', '부빙', '카페키이로', '도트블랭킷', '카페아르크', '대충유원지 인왕산', '카페 하이웨이스트 익선점', '라카페갤러리', '서울서 둘째로잘하는집', '슬로우파크', '내자상회']


In [ ]:
# 셀레니움 이용

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):
    driver.get(url)
    sub_driver.get(url+"/review/visitor")
    time.sleep(3.0)
#     driver.implicitly_wait(10)
#     sub_driver.implicitly_wait(10)

    # driver
    # 네이버 지도의 유형 분류
    try:
        naver_map_type = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#_title > span.DJJvD"))
        )
    except NoSuchElementException:
        print('Element 요소 미존재')
        naver_map_type_list.append('null')
    else:
        driver.implicitly_wait(5)
        print(f'naver_map_type: {naver_map_type.text}')
        naver_map_type_list.append(naver_map_type.text)
        driver.implicitly_wait(5)
    
#     # 네이버 지도의 유형 분류
#     naver_map_type = driver.find_element(By.CSS_SELECTOR, "#_title > span.DJJvD").text
#     print(f'naver_map_type: {naver_map_type}')
#     naver_map_type_list.append(naver_map_type)
#     #time.sleep(2.0)
#     driver.implicitly_wait(5)

    # 블로그 리뷰 수
    try:
#         blog_review_qty = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(3) > a > em"))
#         )
        blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(3) > a > em").text
        if not blog_review_qty:
            print('blog_review_qty ELEMNT does not exist & try other ELEMENT')
            driver.implicitly_wait(5)
#             blog_review_qty = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2) > a > em"))
#             )
            blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2) > a > em").text
    except NoSuchElementException:
        print('Element 요소 미존재')
        blog_review_qty_list.append('null')
    else:
        driver.implicitly_wait(5)
        print(f'블로그 리뷰 수: {blog_review_qty}')
        blog_review_qty_list.append(blog_review_qty)
        #time.sleep(2.0)
        driver.implicitly_wait(5)

    # 별점 점수(2022년 종료 예정)
    try:
#         star_review_stars = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span.PXMot.LXIwF > em"))
#         )
        star_review_stars = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span.PXMot.LXIwF > em").text
        driver.implicitly_wait(5)
        print(f'별점 점수(2022년 종료 예정): {star_review_stars}')
        stars_list.append(star_review_stars)
        #time.sleep(2.0)
    except NoSuchElementException:
        print('Element 요소 미존재')
        stars_list.append('null')



    # sub_driver
    
    # 별점 평가 수(2022년 종료 예정)  //*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]
    try:
#         star_review_qty = WebDriverWait(sub_driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, r'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[3]/span[2]'))
#         )
        star_review_qty = sub_driver.find_element(By.XPATH, r'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[3]/span[2]').text
        if not star_review_qty:
            print('star_review_qty ELEMENT does not exist & try other ELEMENT')
            sub_driver.implicitly_wait(5)
#             star_review_qty = WebDriverWait(sub_driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, r'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]'))
#             )          
            star_review_qty = sub_driver.find_element(By.XPATH, r'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]').text
    except NoSuchElementException:
        print('Element 요소 미존재')
        stars_qty_list.append('null')
    else:
        sub_driver.implicitly_wait(5)
        print(f'별점 평가 수(2022년 종료 예정): {star_review_qty}')
        stars_qty_list.append(star_review_qty)
        #time.sleep(2.0)


    # 방문자 평가 참여자수
    try:
        visitor_review_qty_element = WebDriverWait(sub_driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div._Wmab > span"))
        )
        visitor_review_qty = visitor_review_qty_element.text
        #visitor_review_qty = sub_driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div._Wmab > span").text
        visitor_review_qty = re.sub('[\(\)]', '', visitor_review_qty)
        visitor_review_qty = re.sub('명 참여', '', visitor_review_qty)
        sub_driver.implicitly_wait(5)
        print(f'방문자 평가 참여자수: {visitor_review_qty}')
        visitor_review_qty_list.append(visitor_review_qty)
        #time.sleep(2.0)
    except:
        print(f'방문자 평가 참여자수 10명 미만인 장소')
        visitor_review_qty = '9'
        visitor_review_qty_list.append(visitor_review_qty)


    # 방문자 호평 부분   # k2tmh   ,   uNsI9
    # visitor_eval_list = sub_driver.find_elements(By.CLASS_NAME, "k2tmh")
    # print(f'방문자 호평 부분: {visitor_eval_list}')

    try:
        visitor_eval_list_element = WebDriverWait(sub_driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div.k2tmh > ul"))
        )
        visitor_eval_list = visitor_eval_list_element.text
        #visitor_eval_list = sub_driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div.k2tmh > ul")
        sub_driver.implicitly_wait(5)
        print(f'방문자 호평 부분: {visitor_eval_list}')
        #sub_driver.implicitly_wait(5)
        
        for visitor_eval in visitor_eval_list[:5]:
            print('===========================')
            # 평가 전체 내역
            evaluation = visitor_eval.text
            print(f'evaluation: {evaluation}')
            print('===========================')
            print('for문을 통해 평가 내역 추출')
            evaluation = evaluation.split('\n')
            
            exist_review = []
            
            for i in range(0, 14, 3):
                print(evaluation[i])
                text_evaluation = re.sub('["]', '', evaluation[i])
                print('========'*5)
                print(text_evaluation)

                choose_evaluation = evaluation[i+2]
                print(f'choose_evaluation: {choose_evaluation}')
                print('========'*5)
                
                if text_evaluation in visitor_review_list:
                    dict_review[text_evaluation].append(choose_evaluation)
                    exist_review.append(text_evaluation)

#                 if text_evaluation in visitor_review_list:
#                     dict_review[text_evaluation].append(choose_evaluation)
            for review in visitor_review_list:
                if review not in exist_review:
                    dict_review[review].append('')
        #time.sleep(3.0)
    except:
        for i in visitor_review_list:
            dict_review[i].append('')


#     naver_map_type_list.append(naver_map_type)
#     blog_review_qty_list.append(blog_review_qty)
#     stars_list.append(star_review_stars)
#     stars_qty_list.append(star_review_qty)
#     visitor_review_qty_list.append(visitor_review_qty)

#     # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
#     except Exception as e:
#         print(f"{i}행 문제가 발생: {e}")
        
#         # 내역이 없으므로 null을 임시로 넣어줍니다.
#         naver_map_type_list.append('null')
#         blog_review_qty_list.append('null')
#         stars_list.append('null')
#         stars_qty_list.append('null')
#         visitor_review_qty_list.append('null')
        
driver.quit()
sub_driver.quit()

In [ ]:
print(dict_review)
print(len(dict_review['빵이 맛있어요']))

In [ ]:
print(naver_map_type_list)
print(len(naver_map_type_list))

In [ ]:
print(visitor_review_qty_list)
print(len(visitor_review_qty_list))

In [ ]:
df2['naver_map_type'] = naver_map_type_list            # 네이버 지도의 유형 리스트
df2['blog_review_qty_list'] = blog_review_qty_list     # 블로그 리뷰 수 리스트
df2['stars'] = stars_list                              # 별점 점수(2022년 종료 예정) 정보 리스트
df2['stars_qty'] = stars_qty_list                      # 별점 평가 수(2022년 종료 예정) 리스트
df2['visitor_review_qty'] = visitor_review_qty_list    # 방문자 평가 참여자수 리스트 


for key in dict_review:
    df2[key] = dict_review[key]

In [ ]:
print(stars_qty_list)

In [ ]:
df['naver_map_url'] = ''
df

In [ ]:
df.iloc[0,-1]